# <a target="_blank" href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>


In [52]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)

### Линейная регрессия: практика
Перед началом работы догрузим необходимые нам функции



In [53]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Рассмотрим данные исследования 2004 года о зависимости риска развития диабета от пола, возраста, индекса массы тела, среднего кровяного давления и других показателей. 



#### Загрузка данных

In [54]:
data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

Функция `sklearn.datasets.load_diabetes()` возвращает словарь. В поле `data` записана матрица регрессоров, в которой данные предварительно центрированы и нормированы. В поле `target` записана мера прогрессирования заболевания в течении года. В поле `DESCR` можно прочитать подробнее о данных.

Посмотрим на описание датасета.

In [55]:
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

Поле `data` содержит матрицу размера 442 $\times$ 10, где 442 — количество пациентов, а 10 — количество признаков (возраст, пол, и т.д.). 
Строки матрицы соответствуют пациентам, столбцы — признакам.

In [56]:
data['data'].shape

(442, 10)

Целевая переменная $-$ мера прогрессирования заболевания в течении года.

In [57]:
data['target'].shape

(442,)

Создайте матрицу регрессоров $X$ (data) и столбец наблюдений $y$ (целевая переменная).

In [58]:
X, y = data['data'], data['target']
X, y

(array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
   

#### Обучение моделей

Разбейте данные случайно на две части — обучающую и тестовую в соотношении 80:20.

Если что-то забыли или что-то не понятно, можно почитать справку: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

При разбиении датасета стоит зафиксировать случайность для воспроизводимости результатов, поставив `random_state=42` в функцию разбиения (можете ставить любое число, необязательно именно 42)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(353, 10) (89, 10) (353,) (89,)


Заведите модель линейной регрессии из `sklearn` и обучите ее по обучающей части данных.

In [60]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

LinearRegression()

Посмотрите на результат обучения. Выведите коэффициенты перед признаками и свободный коэффициент.

In [61]:
model.coef_

array([  37.90402135, -241.96436231,  542.42875852,  347.70384391,
       -931.48884588,  518.06227698,  163.41998299,  275.31790158,
        736.1988589 ,   48.67065743])

In [62]:
model.intercept_

151.34560453985995

Выполните предсказание модели на тестовой выборке.

In [63]:
y_pred = model.predict(X_test)

Реализуйте метрики MSE, MAE, MAPE без использования `sklearn` и других готовых реализаций.

Пусть $Y_1, ..., Y_n$ &mdash; истинные значения, а $\widehat{Y}_1, ..., \widehat{Y}_n$ &mdash; предсказания.


Метрика MSE (mean squared error) определяется как
$$MSE = \frac{1}{n}\sum_{i=1}^n \left(Y_i - \widehat{Y}_i\right)^2.$$

Метрика **MAE** (*mean absolute error*), определяемая как 
$$MAE = \frac{1}{n}\sum_{i=1}^n \left|Y_i - \widehat{Y}_i\right|.$$

Метрика **MAPE** (*mean absolute percentage error*), определяемая как 
$$MAPE = 100\% \cdot \frac{1}{n}\sum_{i=1}^n \frac{\left|Y_i - \widehat{Y}_i\right|}{Y_i}.$$


In [64]:
def mean_squared_error(y_true, y_pred):
    return np.sqrt(((y_true - y_pred) ** 2).mean())

def mean_absolute_error(y_true, y_pred):
    return (np.abs(y_true - y_pred)).mean()

def mean_absolute_percentage_error(y_true, y_pred):
    return (np.abs(y_true - y_pred) / y_true).mean() * 100

Посчитайте MSE, MAE, MAPE на тестовой выборке и выведите с точностью до трех знаков после запятой.

In [65]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MSE:\t{mse}\nMAE:\t{mae}\nMAPE:\t{mape}")

MSE:	53.85344583676593
MAE:	42.79409467959994
MAPE:	37.49982636756113


In [66]:
from sklearn import metrics
mse = metrics.mean_squared_error(y_test, y_pred)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)

print(f"MSE:\t{mse ** 0.5}\nMAE:\t{mae}\nMAPE:\t{100 * mape}")

MSE:	53.85344583676593
MAE:	42.79409467959994
MAPE:	37.49982636756113


### Различные разбиения

Разбейте данные на тренировочную и тестовую выборки еще раз: измените пераметр `random_state`. Для каждого из разбиений обучите модель и получите метрики качества. Меняются ли эти метрики, если менять разбиение данных?

In [67]:
mse = []
mae = []
mape = []
for state in [1, 495, 324, 8, 0, 900]:

    # разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)
    
    # обучение модели
    model = LinearRegression(fit_intercept=True)
    model.fit(X_train, y_train)

    # предсказание модели
    y_pred = model.predict(X_test)
    
    # подсчет метрики на тестовой выборке

    mse.append(mean_squared_error(y_test, y_pred))
    mae.append(mean_absolute_error(y_test, y_pred))
    mape.append(mean_absolute_percentage_error(y_test, y_pred))

In [68]:
mse, mae, mape

([54.70449002870804,
  52.866701253532575,
  54.869071969167344,
  54.2021338919284,
  58.517171277315626,
  49.71091007396077],
 [41.97492114949366,
  41.751319833264,
  45.098460608631285,
  44.442055679587284,
  46.1735850037048,
  41.13279719843048],
 [32.49431319195018,
  38.734248537787416,
  41.313918347374816,
  38.989168360820265,
  38.04533165853611,
  40.64797082454018])

**Вывод:**